In [73]:
import numpy as np
import pandas as pd 

In [74]:
import matplotlib.pyplot as plt
import seaborn as sns

In [75]:
from modules import one_hot ,wakachi, meishi,FastText_Vectrizer,rename_section,exam_preprosses

In [76]:
# データの修正

In [77]:
df_train = pd.read_csv('../CSVs/DATA20200401_20220331(TRAIN).csv', encoding='cp932', header=None)
df_test = pd.read_csv('../CSVs/DATA20220401_20220731(TEST).csv', encoding='cp932', header=None)

In [78]:
df_train.head()

,0,1,2,3,4,5,6,7,8
0,37,女,形外,A2,下腿部,長時間手術後、右下腿の激痛を訴えられています。CT にて腓腹筋の腫脹みられます。上記精査お願...,右下腿コンパートメント症候群の疑い,"膝関節から足関節まで含められますか？\r\nSTIR 3方向\r\nT1,T2 sag\r\...",単純
1,0,女,小児,AB1,腰椎(髄),在胎38週5日1954gで出生。臍帯付着部異常（下 腹部）、総排泄腔、膀胱頸部形成不全、鎖肛...,総排泄腔外反症 総排泄腔遺残,"T1, T2, DWI ax\nT1, T2 sag\nFST2 cor\nT2cube",単純
2,0,女,小児,AB1,上腹部(肝胆膵他）,上記精査目的,腹部腫瘤,"T1,T2,DWI,FST2 ax\r\nT1,T2 cor\r\nFST1axまたはcor...",単純＋造影Gd
3,0,男,小児,AB4,泌尿器系下腹部,小陰唇の肥大、癒合傾向あり上記疾患にて精査中 内性器の評価目的です 子宮、卵巣、精巣、精索等...,性分化疾患,"T2 3方向\r\nT1 ax, cor\r\nFST2 ax\r\nDWI ax\r\nT...",単純
4,0,女,小児,B1,肩軟部,在胎38週1日、3147g出生の女児です。2/5で 日齢1になります。左肩に6*7cmの巨大...,血管腫,"T2FS,T1,T2*,DWI ax\r\nT2FS sag,cor\r\n",単純


In [79]:
df_test.head()

,0,1,2,3,4,5,6,7,8
0,96,男,脳外,H1,頭部(委任),T1 T2 FLAIR T2* DWI MRA（頭 首）,左内頚動脈狭窄症,依頼通りお願いします,単純
1,85,男,脳内,A2,下肢血管：動脈（単純）,ASO疑い,閉塞性動脈硬化症の疑い,下肢動脈MRA,単純
2,79,女,皮膚,A2,下肢血管：動脈（単純）,強皮症、末梢動脈疾患による右足趾難治性潰瘍に対してデブリードマン、及び切断術を検討中です。血...,足部難治性皮膚潰瘍,両側下肢MRA （総腸骨動脈～末梢まで）,単純
3,68,女,糖内,A2,下肢血管：動脈（単純）,罹患歴32年の糖尿病血糖管理で入院されている方。両 側のA BI低値みとめており精査目的です。,下肢閉塞性動脈硬化症,全下肢動脈MRA(単純),単純
4,67,男,脳内,A2,下肢血管：動脈（単純）,周期性四肢麻痺（ミオパチー合併タイプ）にて加療中。ABIにて低下あり。下肢動脈狭窄などないで...,下肢閉塞性動脈硬化症の疑い 周期性四肢麻痺,下肢動脈ルーチン,単純


In [80]:
del df_train[7]
del df_test[7]

In [81]:
# ラベルの間違い修正
a=df_train[3].unique()

In [82]:
b=df_test[3].unique()

In [83]:
l1_l2_sym_diff = set(a) ^ set(b)
print(l1_l2_sym_diff)

set()


In [84]:
df_train[df_train.isnull().any(axis=1)]

,0,1,2,3,4,5,6,8


In [85]:
df_test[df_test.isnull().any(axis=1)]

,0,1,2,3,4,5,6,8


In [86]:
#df = df.dropna()

In [87]:
print(len(df_train))

19331


In [88]:
print(len(df_test))

3672


In [89]:
df = pd.concat([df_train, df_test], axis=0)
len(df)

23003

In [90]:
df.loc[df[2].str.contains('総診'), 2] = '救急'
df.loc[df[2].str.contains('ﾍﾟｲﾝ'), 2] = '麻酔'
df.loc[df[2].str.contains('糖内'), 2] = '内泌'

In [91]:
df=df.rename(columns={0:'year',1:'sex',2:'section',3:'label',4:'position',5:'purpose',6:'diagnosis',8:'CE'})

In [92]:
df['label'] = df['label'].replace({'U1':'B1'})

In [93]:
position_list = df['position'].unique()

df['position']=df['position'].replace( list(filter(lambda x: '下肢血管' in x,position_list)), '下肢血管')
df['position']=df['position'].replace( list(filter(lambda x: '頚部' in x,position_list)), '頚部')
df['position']=df['position'].replace( list(filter(lambda x: '上肢' in x,position_list)), '上肢')
df['position']=df['position'].replace( list(filter(lambda x: 'MRdevice' in x,position_list)), 'MRdevice')
df['position']=df['position'].replace( list(filter(lambda x: '全脊椎' in x,position_list)), '全脊椎')
df['position']=df['position'].replace( list(filter(lambda x: '腹部血管' in x,position_list)), '腹部血管')
df['position']=df['position'].replace( list(filter(lambda x: '腎動脈' in x,position_list)), '腹部血管')
df['position']=df['position'].replace( list(filter(lambda x: '下垂体' in x,position_list)), '下垂体')
df['position']=df['position'].replace( list(filter(lambda x: '乳房' in x,position_list)), '乳房')
df['position']=df['position'].replace( list(filter(lambda x: '乳房' in x,position_list)), '乳房')



df['position']=df['position'].replace( ['縦隔','胸部(郭)'], '胸郭')
df['position']=df['position'].replace( ['尿路系MR[VCG(単純)]', '泌尿器系下腹部'], '泌尿器系下腹部')
df['position']=df['position'].replace( ['下腹部委任', '下腹部(一般)'], '下腹部')



In [94]:
df = one_hot(df, 'sex')

['女' '男']


In [95]:
CE_list = df['CE'].unique()
df['CE']=df['CE'].replace( list(filter(lambda x: '造影' in x,CE_list)), '造影')

In [96]:
df = one_hot(df, 'CE')

['単純' '造影']


In [97]:
df_train = df[:19331]
print(len(df_train))

19331


In [98]:
df_test = df[19331:]
print(len(df_test))

3672


In [99]:
#条件にマッチしたIndexを取得
drop_index = df_train.index[(df_train['position'] == 'MRdevice') | 
                      (df_train['position'] == 'WB MRI') | 
                      (df_train['label'] == 'DW') 
                     ]
#drop_index + df.index[df['position'] == 'WB MRI']

print(drop_index)
#条件にマッチしたIndexを削除
df_train = df_train.drop(drop_index)
print(len(df_train))

Int64Index([ 1638,  1639,  1711,  1712,  1713,  1714,  1715,  1716,  1718,
             1791,  2418,  3512,  5352,  5619,  6134,  7587,  7849,  9123,
             9759,  9863,  9908, 10467, 10847, 12353, 12896, 13957, 14178,
            14787, 14792, 15074, 15568, 15895, 16087, 16111, 16171, 16217,
            16218, 16219, 16341, 16697, 17076, 17080, 17210, 17393, 17587,
            18127, 18163, 18309, 18536, 18537, 18566, 18724, 18902, 19271,
            19284],
           dtype='int64')
19276


In [100]:
#条件にマッチしたIndexを取得
drop_index_test = df_test.index[(df_test['position'] == 'MRdevice') | 
                      (df_test['position'] == 'WB MRI') | 
                      (df_test['label'] == 'DW') 
                     ]
#drop_index + df.index[df['position'] == 'WB MRI']

print(drop_index_test)
#条件にマッチしたIndexを削除
df_test = df_test.drop(drop_index_test)
print(len(df_test))

Int64Index([  76,  531,  673,  720,  776,  959, 1254, 1515, 2074, 2232, 2319,
            3179, 3207, 3402, 3503],
           dtype='int64')
3657


In [101]:
a=df_train['position'].value_counts()
b=df_test['position'].value_counts()
pos_count = pd.concat([a,b], axis=1)
pos_count

,position,position
頭部(脳),4673,791
頭部(委任),3607,831
胆道系(MRCP),1577,309
婦人科下腹部,1171,189
腰椎(髄),980,192
上腹部(肝胆膵他）,753,147
下垂体,556,90
頚椎(髄),529,105
膝関節,454,47
MRA脳,394,87


In [102]:
aa = df_train['label'].value_counts()
bb = df_test['label'].value_counts()
label_count = pd.concat([aa,bb],axis=1)
label_count

,label,label
H2,3229,625
H7,1650,308
LI2,1633,320
H6,1102,187
S4,898,180
AB4,783,126
H5,767,113
B1,742,125
H1,687,141
H8,598,115


In [103]:
aaa = df_train['section'].value_counts()
bbb = df_test['section'].value_counts()
sec_count = pd.concat([aaa,bbb],axis=1)
sec_count

,section,section
脳外,4521,870
整形,2558,439
脳内,2301,546
消内,1954,377
母女,1210,189
小児,731,114
呼内,663,87
泌尿,662,130
消外,576,113
耳鼻,569,93


In [104]:
df_train.isnull().all(axis=0)

year         False
section      False
label        False
position     False
purpose      False
diagnosis    False
女            False
男            False
単純           False
造影           False
dtype: bool

In [105]:
df_test.isnull().all(axis=0)

year         False
section      False
label        False
position     False
purpose      False
diagnosis    False
女            False
男            False
単純           False
造影           False
dtype: bool

In [106]:
df_train.to_csv("CorrectedData_train.csv",index=False)
df_test.to_csv("CorrectedData_test2.csv",index=False)

In [107]:
df_test = pd.read_csv("CorrectedData_test2.csv")
df_train = pd.read_csv("CorrectedData_train.csv")

In [108]:
len(df_test)

3657

In [109]:
len(df_train)

19276